In [1]:
import json
from datetime import datetime
import plotly.express as px
from sklearn.model_selection import train_test_split
import sqlite3
import nltk
import spacy
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import umap
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
%matplotlib inline

In [2]:
df_food = pd.read_csv('/Users/robholmstrom/OneDrive/Gitrepos/nonrepo_files/capstone4_files/df_food_clean_rev.csv')

In [4]:
df_food['stars'].replace(3,np.nan, inplace =True)
df_food['stars'].replace(4,np.nan, inplace =True)
df_food.dropna(inplace=True)
df_food['star_binary'] = np.where(df_food['stars'] <2, 'no', 'yes')


In [5]:
df_food['state_star'] = df_food['state'] + df_food['star_binary']


In [6]:
df_food_v2 = df_food.reset_index(drop=True).drop('Unnamed: 0', axis = 1)

In [7]:
df_food_clean = df_food_v2.copy()

In [8]:
#df_food_clean = df_food_v2.loc[df_food_v2['state_star'].str.contains('no')].reset_index(drop=True)


In [9]:
vectorizer = CountVectorizer(analyzer='word', min_df=200, max_features=25)
Xvec = vectorizer.fit_transform(df_food_clean['food_comb'])
bow_df = pd.DataFrame(Xvec.toarray(), columns=vectorizer.get_feature_names())

In [10]:
vectorizertf = TfidfVectorizer(analyzer='word', min_df=50, max_features=50)
Xvectf = vectorizertf.fit_transform(df_food_clean['food_comb'])
tf_df = pd.DataFrame(Xvectf.toarray(), columns=vectorizertf.get_feature_names())

In [11]:
#df_vec = pd.concat([bow_df, df_food_clean['stars_avg']], axis=1)

In [12]:
df_vec = bow_df

In [13]:
df_vec = tf_df

In [14]:
#df_vec = np.array(df_food_clean['stars_avg']).reshape(-1, 1)

In [15]:
X = df_vec
y = df_food_clean['state_star']

In [16]:
X.columns

Index(['american', 'asianfusion', 'bakeries', 'barbeque', 'breakfast',
       'brunch', 'buffets', 'burgers', 'cafes', 'chickenwings', 'chinese',
       'coffee', 'comfortfood', 'desserts', 'diners', 'fastfood', 'french',
       'frozenyogurt', 'gastropubs', 'greek', 'hawaiian', 'italian',
       'japanese', 'juicebars', 'korean', 'mediterranean', 'mexican', 'new',
       'noodles', 'pizza', 'ramen', 'restaurant', 'salad', 'sandwiches',
       'seafood', 'smallplates', 'smoothies', 'soup', 'southern',
       'specialtyfood', 'steakhouses', 'sushibars', 'tacos', 'tapas', 'tea',
       'thai', 'traditional', 'vegan', 'vegetarian', 'vietnamese'],
      dtype='object')

In [17]:
k = 25
if len(X.columns) >= k:
    skb = SelectKBest(k = k, score_func = f_classif)
    best_kfit = skb.fit(X, y)
    X_best_feat = best_kfit.fit_transform(X,y)

    X_best_feat = pd.DataFrame(X_best_feat, columns=X.columns[best_kfit.get_support()])

    best_features = X_best_feat.columns.tolist()
else:
    print('To few columns to select')

In [18]:
best_features

['american',
 'breakfast',
 'brunch',
 'buffets',
 'chickenwings',
 'chinese',
 'comfortfood',
 'fastfood',
 'french',
 'japanese',
 'korean',
 'mexican',
 'new',
 'noodles',
 'pizza',
 'ramen',
 'restaurant',
 'salad',
 'sandwiches',
 'smallplates',
 'southern',
 'steakhouses',
 'tacos',
 'tapas',
 'traditional']

In [19]:
# X = df_vec[best_features]
# y = df_food_clean['state']

In [20]:
X

,american,asianfusion,bakeries,barbeque,breakfast,brunch,buffets,burgers,cafes,chickenwings,...,steakhouses,sushibars,tacos,tapas,tea,thai,traditional,vegan,vegetarian,vietnamese
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.634515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.634515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.634515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.634515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.634515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1256366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1256367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1256368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:

# df_umap = X[(X != 0).any(axis=1)]
# df_umap = X.loc[:, (X != 0).any(axis=0)]
# df_umap

In [ ]:
up = umap.UMAP(n_neighbors = 20, min_dist = .8, metric = 'euclidean')
Xumap = up.fit_transform(X)
sns.scatterplot(Xumap[:, 0], Xumap[:, 1], hue = df_food_clean['star_binary'], s= 10)

In [21]:
le_train = LabelEncoder()
y_le = le_train.fit_transform(y)

In [22]:
np.unique(y_le)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [23]:
y.value_counts()

NVyes    497688
AZyes    397868
NVno     101179
AZno      77738
NCyes     76363
ONyes     75261
ONno      15693
NCno      14580
Name: state_star, dtype: int64

In [24]:
y_state_dict= pd.concat([pd.DataFrame(y_le), pd.DataFrame(y)], axis = 1).groupby('state_star')[0].mean()

In [25]:
y_state_dict

state_star
AZno     0
AZyes    1
NCno     2
NCyes    3
NVno     4
NVyes    5
ONno     6
ONyes    7
Name: 0, dtype: int64

In [26]:
y_state_dict.to_dict()

{'AZno': 0,
 'AZyes': 1,
 'NCno': 2,
 'NCyes': 3,
 'NVno': 4,
 'NVyes': 5,
 'ONno': 6,
 'ONyes': 7}

In [27]:
y_state_dict_swap = {value:key for key, value in y_state_dict.items()}
y_state_dict_swap

{0: 'AZno',
 1: 'AZyes',
 2: 'NCno',
 3: 'NCyes',
 4: 'NVno',
 5: 'NVyes',
 6: 'ONno',
 7: 'ONyes'}

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y_le, stratify = y_le, random_state = 42)

In [29]:
y_train

array([1, 1, 5, ..., 5, 1, 4])

In [30]:
rus = RandomUnderSampler()
X_train_rus, y_train_rus = rus.fit_sample(X_train, y_train)
X_test_rus, y_test_rus = rus.fit_sample(X_test, y_test)


In [31]:
y_train

array([1, 1, 5, ..., 5, 1, 4])

In [32]:

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_rus)
X_test_scaled = scaler.transform(X_test_rus)

In [33]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class= 'ovr', solver = 'lbfgs', penalty = 'l2', max_iter = 1000)

In [34]:
lr.fit(X_train_scaled, y_train_rus)

LogisticRegression(max_iter=1000, multi_class='ovr')

In [35]:
y_test_pred = lr.predict(X_test_scaled)
y_train_pred = lr.predict(X_train_scaled)

In [36]:
y_train_pred

array([1, 1, 0, ..., 7, 0, 3])

In [37]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,log_loss, precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
print('LR1 confusion matrix')
#lr1train = confusion_matrix(y_train, y_pred_train, normalize= None)
lr1test = confusion_matrix(y_test_rus, y_test_pred, normalize = None)
pd.DataFrame(lr1test).rename(columns = y_state_dict_swap, index = y_state_dict_swap)

LR1 confusion matrix


,AZno,AZyes,NCno,NCyes,NVno,NVyes,ONno,ONyes
AZno,705,819,695,249,265,199,432,281
AZyes,475,1110,512,281,210,281,412,364
NCno,389,497,1146,573,134,181,322,403
NCyes,278,575,689,823,151,263,324,542
NVno,375,452,586,222,611,308,479,612
NVyes,381,575,464,277,380,495,428,645
ONno,157,281,301,285,184,191,1265,981
ONyes,129,363,244,303,151,258,916,1281


In [38]:
print('Test macro average recall score is ',recall_score(y_test_rus, y_test_pred, average = 'macro'))
print('Train macro average recall score is ',recall_score(y_train_rus, y_train_pred, average = 'macro'))

Test macro average recall score is  0.2550068587105624
Train macro average recall score is  0.25850480109739365


In [39]:
print('Test accuracy score is ',accuracy_score(y_test_rus, y_test_pred))
print('Train accuracy score is ',accuracy_score(y_train_rus, y_train_pred))

Test accuracy score is  0.2550068587105624
Train accuracy score is  0.2585048010973937


In [40]:
recall_score(y_test_rus, y_test_pred, average = 'macro')/(1/lr1test.shape[0])

2.040054869684499

In [41]:
knn1 = KNeighborsClassifier(n_neighbors=15, weights='uniform', leaf_size=20)
knn1.fit(X_train_scaled, y_train_rus)

y_test_predknn1 = knn1.predict(X_test_scaled)
y_train_predknn1 = knn1.predict(X_train_scaled)
print('Test macro average recall score is ',recall_score(y_test_rus, y_test_predknn1,average = 'macro'))
print('Train macro average recall score is ',recall_score(y_train_rus, y_train_predknn1,average = 'macro'))

Test macro average recall score is  0.3087105624142661
Train macro average recall score is  0.32445130315500686


In [42]:
knntest = confusion_matrix(y_test_rus, y_test_pred, normalize = None)
pd.DataFrame(knntest).rename(columns = y_state_dict_swap)

,AZno,AZyes,NCno,NCyes,NVno,NVyes,ONno,ONyes
0,705,819,695,249,265,199,432,281
1,475,1110,512,281,210,281,412,364
2,389,497,1146,573,134,181,322,403
3,278,575,689,823,151,263,324,542
4,375,452,586,222,611,308,479,612
5,381,575,464,277,380,495,428,645
6,157,281,301,285,184,191,1265,981
7,129,363,244,303,151,258,916,1281


In [43]:
print('Test accuracy score is ',accuracy_score(y_test_rus, y_test_predknn1))
print('Train accuracy score is ',accuracy_score(y_train_rus, y_train_predknn1))

Test accuracy score is  0.30871056241426614
Train accuracy score is  0.32445130315500686


In [44]:
rf = RandomForestClassifier(max_depth = 8)
rf.fit(X_train_scaled, y_train_rus)
y_test_predrf = rf.predict(X_test_scaled)
y_train_predrf = rf.predict(X_train_scaled)

print('Test macro average recall score is ',recall_score(y_test_rus, y_test_predrf,average = 'macro'))
print('Train macro average recall score is ',recall_score(y_train_rus, y_train_predrf,average = 'macro'))

Test macro average recall score is  0.3052469135802469
Train macro average recall score is  0.3109510745313214


In [45]:
rftest = confusion_matrix(y_test_rus, y_test_predrf, normalize = None)
pd.DataFrame(rftest).rename(columns = y_state_dict_swap)

,AZno,AZyes,NCno,NCyes,NVno,NVyes,ONno,ONyes
0,905,747,585,108,94,85,160,961
1,589,1089,412,123,70,98,146,1118
2,325,278,1231,586,32,52,133,1008
3,246,360,680,950,39,48,92,1230
4,394,274,457,126,606,302,217,1269
5,348,299,348,179,232,715,144,1380
6,164,159,260,63,68,33,922,1976
7,164,205,192,59,38,25,479,2483
